## Analyzing 2019 CPS Data 

This notebook contains the code used to wrangle, join, and analyze the November 2019 CURRENT POPULATION SURVEY (CPS) [Computer and Internet Use Supplement](https://www.census.gov/data/datasets/time-series/demo/cps/cps-supp_cps-repwgt/cps-computer.html).

All csv files can be found in the "data" folder of the [working](https://github.com/danielgrzenda/broadbandequity/tree/working) branch of our Broadband Equity Github repo.

##### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

##### Fetching API data

In [ ]:
#from data_pipeline import fetch_census_data

In [ ]:
#fetch_census_data.cps_individual(force_api_call=True)

##### Loading dataset

In [4]:
# loading cps data

census_df=pd.read_csv("data/cps_individual.csv").drop(
                        ['Unnamed: 0', 'state', 'county'], axis=1)

In [6]:
# filtering for Chicago
# 562 rows x 26 columns
# -1 = yes
# 2 = no

census_df = census_df[(census_df['area'] == 16980) & 
                      (census_df['city'] == 1)]

census_df.head(5)

,area,city,use_internet,hh_id,hh_size,employment,education,hispanic,race,internet_provider,...,cant_afford_internet,internet_NA,no_device_internet,other_no_internet,doctor_internet,health_research_online,health_monitor_online,health_records_online,buy_internet_lower_price,online_classes
31,16980,1,1,310604531644537,3,7,43,2,4,1,...,-1,-1,-1,-1,2,2,2,2,-1,1
32,16980,1,1,310604531644537,3,7,43,2,4,1,...,-1,-1,-1,-1,2,2,2,2,-1,-1
33,16980,1,1,310604531644537,3,1,44,2,4,1,...,-1,-1,-1,-1,2,2,2,2,-1,-1
34,16980,1,1,107163554710898,2,2,43,2,1,1,...,-1,-1,-1,-1,2,1,2,2,-1,-1
35,16980,1,1,107163554710898,2,3,44,2,1,1,...,-1,-1,-1,-1,2,1,2,2,-1,1


##### Demographics

In [7]:
# first we look at the overall demographics of the sample size

demo_df = census_df[[
                'hh_id',
                'hh_size',
                'employment',
                'education',
                'hispanic',
                'race']]

demo_df

,hh_id,hh_size,employment,education,hispanic,race
31,310604531644537,3,7,43,2,4
32,310604531644537,3,7,43,2,4
33,310604531644537,3,1,44,2,4
34,107163554710898,2,2,43,2,1
35,107163554710898,2,3,44,2,1
...,...,...,...,...,...,...
3146,617768301395103,4,1,39,1,1
3147,617768301395103,4,1,39,1,1
3148,617768301395103,4,1,43,1,1
3149,617768301395103,4,7,36,1,1


In [19]:
# -1 = NA in employment/edu columns. replace with NA
# group employment to 1,2,3 (employed, unemployed, other)
# group education to 1,2,3 (<= high school, some college, bachelors+)
# 

demo_df.race.unique()

array([4, 1, 2, 3, 8, 5, 6])

In [ ]:
def my_recode(column):
    if column == -1:
        return 0
    else:
        return 1

##### Internet Access

##### Internet Usage